In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.stat import Correlation
#import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
#import holidays
from datetime import datetime, timezone
from pyspark.ml.classification import RandomForestClassifier, BinaryLogisticRegressionSummary
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.pipeline import PipelineModel

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("US_Accidents") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR") #supress warnings

/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/11/26 15:51:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/26 15:51:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/26 15:51:25 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/26 15:51:25 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/11/26 15:51:25 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/11/26 15:51:25 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
24/11/26 15:51:25 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
24/11/26 15:51:25 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
24/11/26 15:51:25 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
24/11/26 15:51:25 WARN Utils: Service 'SparkUI' could not bind on

In [3]:
# Read in Data
df = spark.read.parquet("updated_dataset.parquet")

In [4]:
# Use StringIndexer for encoding the 'Severity' column
indexer = StringIndexer(inputCol="Severity", outputCol="SeverityIndex")
df = indexer.fit(df).transform(df)

In [5]:
# Create list of features
feature_list = []
for col in df.columns:
    if col == 'Severity':
        continue
    elif col == 'SeverityIndex':
        continue
    elif col == 'Severity_Binary':
        continue
    else:
        feature_list.append(col)

In [6]:
#feature_list = ['Temperature', 'Pressure', 'Astronomical_TwilightIndex', 'Interstate_Indicator', 'Rush_Hour', 'SeasonVec', 'Traffic_Intersection', 'Urban_Ratio', 'Wind_Speed']

In [6]:
# Split the data into train and test
splits = df.randomSplit([0.8, 0.2], 314)
train = splits[0]
test = splits[1]

In [7]:
# Undersampling
from pyspark.sql import functions as F

# Step 1: Group by 'Severity' and count occurrences
class_counts = train.groupBy("SeverityIndex").count()

# Step 2: Use PySpark's min() function to find the minimum count
min_class_size = class_counts.agg(F.min('count')).collect()[0][0]

undersampled_train_list = []

for row in class_counts.collect():
    class_label = row['SeverityIndex']
    class_size = row['count']

    if class_size > min_class_size:
        # Sample the data for this class to the size of the minimum class
        class_data = train.filter(F.col("SeverityIndex") == class_label)
        class_data_undersampled = class_data.sample(withReplacement=False, fraction=min_class_size / class_size)
    else:
        # For classes that are already at the minimum size, keep all samples
        class_data_undersampled = train.filter(F.col("SeverityIndex") == class_label)

    undersampled_train_list.append(class_data_undersampled)

# Combine all the undersampled DataFrames
undersampled_train = undersampled_train_list[0]  # start with the first one
for df in undersampled_train_list[1:]:
    undersampled_train = undersampled_train.union(df)

# Show the result
undersampled_train.show()

# Step 4: Group by 'Severity' and count the occurrences in the undersampled DataFrame
undersampled_class_counts = undersampled_train.groupBy("SeverityIndex").count()

# Show the result
undersampled_class_counts.show()

+--------+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+-------------+
|Severity|Temperature|Humidity|Pressure|Visibility|Wind_Speed|Precipitation|Weekday|Rush_Hour|Rain|Snow|    SeasonVec|Astronomical_TwilightIndex|Interstate_Indicator|Sex_ratio|Percent_Age_65_over|MedianIncome|Urban_Ratio|Traffic_Interference|Traffic_Intersection|Destination|Percent_Age_15-24|Severity_Binary|SeverityIndex|
+--------+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+-------------+
|       2|        -15|      

+-------------+-----+
|SeverityIndex|count|
+-------------+-----+
|          0.0|51939|
|          1.0|52447|
|          3.0|52151|
|          2.0|51986|
+-------------+-----+



In [8]:
# Assemble data for logistic regression model
assembler = VectorAssembler(inputCols=feature_list,
                            outputCol="features")

undersampled_train = assembler.transform(undersampled_train)
test = assembler.transform(test)

In [9]:
# Standardize the predictors
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(undersampled_train)
scaledTrainData = scalerModel.transform(undersampled_train)
scaledTestData = scalerModel.transform(test)

In [10]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("10-Fold Cross Validation Example") \
    .getOrCreate()

# Define the model and set the label column
lr = LogisticRegression(featuresCol="scaledFeatures", labelCol="SeverityIndex", family="multinomial")

# Define the hyperparameter grid
param_grid = ParamGridBuilder() \
    .addGrid(lr.maxIter, [50, 100, 150]) \
    .addGrid(lr.regParam, [0.0, 0.1, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

# Define the evaluator
evaluator = MulticlassClassificationEvaluator(labelCol="SeverityIndex", metricName="accuracy")

# Define the CrossValidator
cross_validator = CrossValidator(
    estimator=lr,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    numFolds=10,  #for 10-fold cross-validation
    parallelism=4,  #number of threads for parallelism
    seed = 314
)

# Perform cross-validation
cv_model = cross_validator.fit(scaledTrainData)

# Print the best hyperparameters
best_model = cv_model.bestModel
print(f"Best Parameters: maxIter={best_model._java_obj.getMaxIter()}, "
      f"regParam={best_model._java_obj.getRegParam()}, "
      f"elasticNetParam={best_model._java_obj.getElasticNetParam()}")

Best Parameters: maxIter=50, regParam=0.0, elasticNetParam=0.0


In [11]:
# Fit logistic regression model with intercept
from pyspark.ml.classification import LogisticRegression

# instantiate the model
lr = LogisticRegression(labelCol='SeverityIndex',
                        featuresCol='scaledFeatures',
                        maxIter=50, 
                        regParam=0.0, 
                        elasticNetParam=0.0,
                        family="multinomial")

# Fit the model
lrModel = lr.fit(scaledTrainData)
print(f"Detected number of classes: {lrModel.numClasses}")

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

Detected number of classes: 4
Coefficients: DenseMatrix([[-0.01489983, -0.07676855,  0.03117155, -0.03029891, -0.00874672,
              -0.00169415, -0.03478702, -0.03807731, -0.01306237,  0.04263927,
               0.54222826,  0.33835008,  0.03013158, -0.0050059 , -0.10234176,
               0.03324799,  0.03142831,  0.00884773,  0.04310394, -0.01489743,
              -0.0296616 ,  0.06551928,  0.00583615],
             [-0.14508323, -0.08339894,  0.21737174, -0.00554761,  0.12928795,
               0.01082213, -0.06374609, -0.01297396,  0.05168738,  0.02983143,
               0.21099112,  0.15029372, -0.1292759 , -0.11035717,  0.46458172,
               0.05128523, -0.12927579,  0.0044287 ,  0.2848914 ,  0.01295158,
              -0.25923277, -0.10363481, -0.02557472],
             [-0.24953668, -0.0461124 ,  0.02180161,  0.02661451,  0.0632119 ,
              -0.00474235, -0.14067781, -0.17343659,  0.01350265,  0.03925907,
               0.21793878,  0.14634185, -0.0753478 ,  0.16

In [12]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# compute predictions. this will append column "prediction" to dataframe
lrPred = lrModel.transform(scaledTestData)
lrPred.select("prediction").distinct().show()

evaluator = MulticlassClassificationEvaluator(labelCol='SeverityIndex', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(lrPred, {evaluator.metricName: "accuracy"})
print(f'Accuracy: {accuracy}')

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
|       3.0|
|       2.0|
+----------+



Accuracy: 0.3860610561478687


In [13]:
precision = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedPrecision'})
recall = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedRecall'})
f1_score = evaluator.evaluate(lrPred, {evaluator.metricName: 'f1'})

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')

Precision: 0.767945209738235
Recall: 0.3860610561478686
F1 Score: 0.47636683203213875


In [14]:
# Add a column to indicate correct or incorrect predictions
predictions = lrPred.withColumn(
    'is_correct', F.expr("CASE WHEN SeverityIndex = prediction THEN 1 ELSE 0 END")
)

# Calculate accuracy by class
accuracy_by_class = predictions.groupBy('SeverityIndex').agg(
    (F.sum('is_correct') / F.count('SeverityIndex')).alias('accuracy')
)

# Show per-class accuracy
accuracy_by_class.show()

+-------------+-------------------+
|SeverityIndex|           accuracy|
+-------------+-------------------+
|          0.0|0.35746522361957145|
|          1.0| 0.5007498180914689|
|          3.0| 0.7780001539527365|
|          2.0|0.42563229300429906|
+-------------+-------------------+



In [15]:
# Calculate metrics by class
labels = lrPred.select('SeverityIndex').distinct().orderBy('SeverityIndex').rdd.flatMap(lambda x: x).collect()

metrics = {}
for label in labels:
    # Filter predictions for the current label
    true_positive = lrPred.filter((F.col('SeverityIndex') == label) & (F.col('prediction') == label)).count()
    false_positive = lrPred.filter((F.col('SeverityIndex') != label) & (F.col('prediction') == label)).count()
    false_negative = lrPred.filter((F.col('SeverityIndex') == label) & (F.col('prediction') != label)).count()
    true_negative = lrPred.filter((F.col('SeverityIndex') != label) & (F.col('prediction') != label)).count()

    # Precision, Recall, and F1 Score
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    # Store metrics
    metrics[label] = {'precision': precision, 'recall': recall, 'f1_score': f1_score}

# Print metrics for each class
for label, metric in metrics.items():
    print(f"Class {label} - Precision: {metric['precision']:.4f}, Recall: {metric['recall']:.4f}, F1 Score: {metric['f1_score']:.4f}")

Class 0.0 - Precision: 0.8876, Recall: 0.3575, F1 Score: 0.5097
Class 1.0 - Precision: 0.3219, Recall: 0.5007, F1 Score: 0.3919
Class 2.0 - Precision: 0.0623, Recall: 0.4256, F1 Score: 0.1087
Class 3.0 - Precision: 0.0285, Recall: 0.7780, F1 Score: 0.0550
